# Phase 3: Advanced Modeling & Experimentation

**Objective**: Beat the baseline with ML models, feature engineering, and hyperparameter tuning. Includes anomaly detection and model interpretability.

## Table of Contents
1. Setup & Data Loading
2. Anomaly Detection (Isolation Forest, Z-score)
3. Feature Engineering
4. Model Training (XGBoost, Prophet)
5. Hyperparameter Optimization (Optuna)
6. Feature Importance Analysis
7. Residual Analysis
8. Model Comparison (vs Baseline)
9. MLflow Logging & Champion Model Selection
10. Save Champion Model

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import IsolationForest
from scipy import stats
import xgboost as xgb
from prophet import Prophet
import optuna
import mlflow
import mlflow.sklearn
import joblib
from pathlib import Path
import warnings
import logging

warnings.filterwarnings('ignore')
optuna.logging.set_verbosity(optuna.logging.WARNING)
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Visualization config
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 11

# Paths
PROJECT_ROOT = Path("..").resolve()
PROCESSED_DIR = PROJECT_ROOT / "Data" / "processed"
RESULTS_DIR = PROJECT_ROOT / "results"
MODELS_DIR = PROJECT_ROOT / "models"
RESULTS_DIR.mkdir(exist_ok=True)
MODELS_DIR.mkdir(exist_ok=True)

def mean_absolute_percentage_error(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    """Calculate MAPE, handling zeros."""
    mask = y_true != 0
    if mask.sum() == 0:
        return np.nan
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

# Load cleaned data
data_path = PROCESSED_DIR / "daily_sales_clean.parquet"
if data_path.exists():
    df = pd.read_parquet(data_path)
    print(f"✅ Loaded cleaned data from: {data_path}")
else:
    data_path = PROCESSED_DIR / "daily_sales.parquet"
    df = pd.read_parquet(data_path)
    print(f"⚠️ Loaded original data from: {data_path}")

df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date').sort_index()

# Load baseline results for comparison
baseline_path = RESULTS_DIR / "baseline_results.csv"
if baseline_path.exists():
    baseline_results = pd.read_csv(baseline_path)
    best_baseline_mae = baseline_results['MAE'].min()
    print(f"📊 Best baseline MAE: {best_baseline_mae:,.0f} PLN")
else:
    best_baseline_mae = None
    print("⚠️ No baseline results found")

print(f"\nData shape: {df.shape}")
print(f"Date range: {df.index.min().date()} to {df.index.max().date()}")
df.head()

## 2. Anomaly Detection

Detecting anomalies using multiple methods:
- **Isolation Forest**: Unsupervised ML approach
- **Z-score**: Statistical threshold method
- **IQR**: Interquartile range method

In [ ]:
# ============================================
# ANOMALY DETECTION
# ============================================

# Only analyze non-zero sales for anomaly detection
sales_data = df[df['sales'] > 0]['sales'].values.reshape(-1, 1)

# Method 1: Isolation Forest
iso_forest = IsolationForest(contamination=0.05, random_state=42, n_estimators=100)
df['anomaly_isoforest'] = 0
df.loc[df['sales'] > 0, 'anomaly_isoforest'] = iso_forest.fit_predict(sales_data)
df['anomaly_isoforest'] = (df['anomaly_isoforest'] == -1).astype(int)

# Method 2: Z-score (threshold = 3)
z_scores = np.zeros(len(df))
nonzero_mask = df['sales'] > 0
z_scores[nonzero_mask] = np.abs(stats.zscore(df.loc[nonzero_mask, 'sales']))
df['z_score'] = z_scores
df['anomaly_zscore'] = (df['z_score'] > 3).astype(int)

# Method 3: IQR
Q1 = df.loc[nonzero_mask, 'sales'].quantile(0.25)
Q3 = df.loc[nonzero_mask, 'sales'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
df['anomaly_iqr'] = ((df['sales'] > upper_bound) | ((df['sales'] < lower_bound) & (df['sales'] > 0))).astype(int)

# Combined anomaly flag (any method flags it)
df['anomaly_any'] = ((df['anomaly_isoforest'] == 1) | (df['anomaly_zscore'] == 1) | (df['anomaly_iqr'] == 1)).astype(int)

# Summary
print("="*50)
print("ANOMALY DETECTION RESULTS")
print("="*50)
print(f"\nIsolation Forest: {df['anomaly_isoforest'].sum()} anomalies ({100*df['anomaly_isoforest'].mean():.1f}%)")
print(f"Z-score (>3):     {df['anomaly_zscore'].sum()} anomalies ({100*df['anomaly_zscore'].mean():.1f}%)")
print(f"IQR method:       {df['anomaly_iqr'].sum()} anomalies ({100*df['anomaly_iqr'].mean():.1f}%)")
print(f"Combined (any):   {df['anomaly_any'].sum()} anomalies ({100*df['anomaly_any'].mean():.1f}%)")

# Visualize anomalies
fig, axes = plt.subplots(2, 1, figsize=(16, 8))

# Full time series with anomalies
axes[0].plot(df.index, df['sales'], 'b-', linewidth=0.8, alpha=0.7, label='Sales')
anomaly_dates = df[df['anomaly_any'] == 1].index
anomaly_values = df.loc[anomaly_dates, 'sales']
axes[0].scatter(anomaly_dates, anomaly_values, c='red', s=50, label='Anomalies', zorder=5)
axes[0].axhline(y=upper_bound, color='orange', linestyle='--', alpha=0.5, label=f'IQR Upper: {upper_bound:,.0f}')
axes[0].set_title("Daily Sales with Detected Anomalies")
axes[0].set_ylabel("Sales (PLN)")
axes[0].legend()

# Anomaly score distribution (Isolation Forest)
iso_scores = iso_forest.decision_function(sales_data)
axes[1].hist(iso_scores, bins=50, edgecolor='black', alpha=0.7)
axes[1].axvline(x=0, color='red', linestyle='--', label='Anomaly threshold')
axes[1].set_title("Isolation Forest Anomaly Scores")
axes[1].set_xlabel("Anomaly Score (negative = more anomalous)")
axes[1].legend()

plt.tight_layout()
plt.show()

# Show top anomalies
print("\nTop 10 Anomalous Days (by sales value):")
top_anomalies = df[df['anomaly_any'] == 1].nlargest(10, 'sales')[['sales', 'anomaly_isoforest', 'anomaly_zscore', 'anomaly_iqr']]
for idx, row in top_anomalies.iterrows():
    methods = []
    if row['anomaly_isoforest']: methods.append('IsoForest')
    if row['anomaly_zscore']: methods.append('Z-score')
    if row['anomaly_iqr']: methods.append('IQR')
    print(f"  {idx.strftime('%Y-%m-%d')} ({idx.strftime('%A'):9s}): {row['sales']:>10,.0f} PLN - {', '.join(methods)}")

## 3. Feature Engineering

Creating features for ML models:
- Calendar features (day of week, month, cyclic encoding)
- Lag features (lag 1, 7, 14)
- Rolling statistics (mean, std)

In [ ]:
# ============================================
# FEATURE ENGINEERING
# ============================================

# Calendar features
if 'weekday' not in df.columns:
    df['weekday'] = df.index.dayofweek
df['month'] = df.index.month
df['day_of_year'] = df.index.dayofyear
df['week_of_year'] = df.index.isocalendar().week.astype(int)
df['is_weekend'] = (df['weekday'] >= 5).astype(int)

# Cyclic encoding for periodicity
df['weekday_sin'] = np.sin(2 * np.pi * df['weekday'] / 7)
df['weekday_cos'] = np.cos(2 * np.pi * df['weekday'] / 7)
df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
df['day_sin'] = np.sin(2 * np.pi * df['day_of_year'] / 365)
df['day_cos'] = np.cos(2 * np.pi * df['day_of_year'] / 365)

# Lag features
for lag in [1, 7, 14, 21, 28]:
    df[f'lag_{lag}'] = df['sales'].shift(lag)

# Rolling statistics
for window in [7, 14, 28]:
    df[f'rolling_mean_{window}'] = df['sales'].shift(1).rolling(window=window).mean()
    df[f'rolling_std_{window}'] = df['sales'].shift(1).rolling(window=window).std()
    df[f'rolling_min_{window}'] = df['sales'].shift(1).rolling(window=window).min()
    df[f'rolling_max_{window}'] = df['sales'].shift(1).rolling(window=window).max()

# Expanding mean (all historical data)
df['expanding_mean'] = df['sales'].shift(1).expanding().mean()

# Drop rows with NaN from lag/rolling features
df_model = df.dropna().copy()

print(f"Features created: {df_model.shape[1]} columns")
print(f"Data points available for modeling: {len(df_model)}")
print(f"\nFeature list:")
feature_cols = [c for c in df_model.columns if c not in ['sales', 'order_count', 'avg_order_value', 
                                                          'anomaly_isoforest', 'anomaly_zscore', 
                                                          'anomaly_iqr', 'anomaly_any', 'z_score', 'is_outlier']]
for i, col in enumerate(feature_cols, 1):
    print(f"  {i:2d}. {col}")

## 4. Model Training

Training XGBoost and Prophet models on the prepared data.

In [ ]:
# ============================================
# TRAIN/TEST SPLIT
# ============================================
train_size = int(len(df_model) * 0.8)
train = df_model.iloc[:train_size]
test = df_model.iloc[train_size:]

# Define features (exclude target and metadata columns)
exclude_cols = ['sales', 'order_count', 'avg_order_value', 'anomaly_isoforest', 
                'anomaly_zscore', 'anomaly_iqr', 'anomaly_any', 'z_score', 'is_outlier']
features = [c for c in df_model.columns if c not in exclude_cols]

X_train = train[features]
y_train = train['sales']
X_test = test[features]
y_test = test['sales']

print(f"Train: {len(train)} samples")
print(f"Test:  {len(test)} samples")
print(f"Features: {len(features)}")

# Store results
results = []
predictions = {}

# ============================================
# XGBOOST (default params)
# ============================================
print("\n" + "="*50)
print("Training XGBoost (default params)...")
print("="*50)

xgb_model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    random_state=42
)
xgb_model.fit(X_train, y_train)
xgb_pred = xgb_model.predict(X_test)

xgb_mae = mean_absolute_error(y_test, xgb_pred)
xgb_rmse = np.sqrt(mean_squared_error(y_test, xgb_pred))
xgb_mape = mean_absolute_percentage_error(y_test.values, xgb_pred)

results.append({'Model': 'XGBoost (default)', 'MAE': xgb_mae, 'RMSE': xgb_rmse, 'MAPE': xgb_mape})
predictions['XGBoost (default)'] = xgb_pred
print(f"XGBoost: MAE={xgb_mae:,.0f}, RMSE={xgb_rmse:,.0f}, MAPE={xgb_mape:.1f}%")

# ============================================
# PROPHET
# ============================================
print("\n" + "="*50)
print("Training Prophet...")
print("="*50)

# Prepare data for Prophet
prophet_train = train.reset_index()[['date', 'sales']].rename(columns={'date': 'ds', 'sales': 'y'})
prophet_test = test.reset_index()[['date', 'sales']].rename(columns={'date': 'ds', 'sales': 'y'})

prophet_model = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=False
)
prophet_model.fit(prophet_train)

future = prophet_model.make_future_dataframe(periods=len(prophet_test))
forecast = prophet_model.predict(future)
prophet_pred = forecast['yhat'].iloc[-len(prophet_test):].values

prophet_mae = mean_absolute_error(prophet_test['y'], prophet_pred)
prophet_rmse = np.sqrt(mean_squared_error(prophet_test['y'], prophet_pred))
prophet_mape = mean_absolute_percentage_error(prophet_test['y'].values, prophet_pred)

results.append({'Model': 'Prophet', 'MAE': prophet_mae, 'RMSE': prophet_rmse, 'MAPE': prophet_mape})
predictions['Prophet'] = prophet_pred
print(f"Prophet: MAE={prophet_mae:,.0f}, RMSE={prophet_rmse:,.0f}, MAPE={prophet_mape:.1f}%")

## 5. Hyperparameter Optimization with Optuna

Tuning XGBoost hyperparameters using Bayesian optimization.

In [ ]:
# ============================================
# OPTUNA HYPERPARAMETER OPTIMIZATION
# ============================================
print("="*50)
print("Running Optuna optimization (20 trials)...")
print("="*50)

def objective(trial):
    """Optuna objective function for XGBoost tuning."""
    params = {
        'objective': 'reg:squarederror',
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'random_state': 42
    }
    
    model = xgb.XGBRegressor(**params)
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    return mean_absolute_error(y_test, pred)

# Run optimization
study = optuna.create_study(direction='minimize', study_name='xgboost_tuning')
study.optimize(objective, n_trials=20, show_progress_bar=True)

print(f"\nBest MAE: {study.best_value:,.0f}")
print(f"Best params: {study.best_params}")

# Train final model with best params
best_params = study.best_params
best_params['objective'] = 'reg:squarederror'
best_params['random_state'] = 42

best_xgb = xgb.XGBRegressor(**best_params)
best_xgb.fit(X_train, y_train)
best_xgb_pred = best_xgb.predict(X_test)

best_mae = mean_absolute_error(y_test, best_xgb_pred)
best_rmse = np.sqrt(mean_squared_error(y_test, best_xgb_pred))
best_mape = mean_absolute_percentage_error(y_test.values, best_xgb_pred)

results.append({'Model': 'XGBoost (tuned)', 'MAE': best_mae, 'RMSE': best_rmse, 'MAPE': best_mape})
predictions['XGBoost (tuned)'] = best_xgb_pred
print(f"\nXGBoost (tuned): MAE={best_mae:,.0f}, RMSE={best_rmse:,.0f}, MAPE={best_mape:.1f}%")

# Optuna visualization - these functions create their own figures
# Optimization history
fig_history = optuna.visualization.matplotlib.plot_optimization_history(study)
fig_history.figure.set_size_inches(14, 4)
plt.title("Optuna Optimization History")
plt.tight_layout()
plt.show()

# Parameter importance
try:
    fig_importance = optuna.visualization.matplotlib.plot_param_importances(study)
    fig_importance.figure.set_size_inches(14, 4)
    plt.title("Hyperparameter Importance")
    plt.tight_layout()
    plt.show()
except Exception as e:
    print(f"Could not plot parameter importance: {e}")

## 6. Feature Importance Analysis

Understanding which features contribute most to the XGBoost predictions.

In [ ]:
# ============================================
# FEATURE IMPORTANCE
# ============================================

# Get feature importance from tuned XGBoost
importance_df = pd.DataFrame({
    'feature': features,
    'importance': best_xgb.feature_importances_
}).sort_values('importance', ascending=False)

print("="*50)
print("TOP 15 MOST IMPORTANT FEATURES")
print("="*50)
print(importance_df.head(15).to_string(index=False))

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Bar chart - top 15
top_features = importance_df.head(15)
axes[0].barh(range(len(top_features)), top_features['importance'].values, color='steelblue')
axes[0].set_yticks(range(len(top_features)))
axes[0].set_yticklabels(top_features['feature'].values)
axes[0].invert_yaxis()
axes[0].set_xlabel('Importance')
axes[0].set_title('Top 15 Feature Importance (XGBoost)')

# Cumulative importance
importance_df['cumulative'] = importance_df['importance'].cumsum() / importance_df['importance'].sum()
axes[1].plot(range(len(importance_df)), importance_df['cumulative'].values, 'b-', linewidth=2)
axes[1].axhline(y=0.8, color='red', linestyle='--', label='80% threshold')
axes[1].axhline(y=0.95, color='orange', linestyle='--', label='95% threshold')
axes[1].set_xlabel('Number of Features')
axes[1].set_ylabel('Cumulative Importance')
axes[1].set_title('Cumulative Feature Importance')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Find features that explain 80% and 95% of importance
n_80 = (importance_df['cumulative'] <= 0.8).sum() + 1
n_95 = (importance_df['cumulative'] <= 0.95).sum() + 1
print(f"\nFeatures needed for 80% importance: {n_80}")
print(f"Features needed for 95% importance: {n_95}")

## 7. Residual Analysis

Examining prediction errors to understand model behavior and identify potential improvements.

In [ ]:
# ============================================
# RESIDUAL ANALYSIS (XGBoost Tuned)
# ============================================

residuals = y_test.values - best_xgb_pred

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Residuals over time
axes[0, 0].plot(test.index, residuals, 'b-', linewidth=0.8)
axes[0, 0].axhline(y=0, color='red', linestyle='--')
axes[0, 0].fill_between(test.index, residuals, 0, alpha=0.3)
axes[0, 0].set_title('Residuals Over Time')
axes[0, 0].set_xlabel('Date')
axes[0, 0].set_ylabel('Residual (Actual - Predicted)')

# 2. Residual distribution
axes[0, 1].hist(residuals, bins=30, edgecolor='black', alpha=0.7)
axes[0, 1].axvline(x=0, color='red', linestyle='--')
axes[0, 1].axvline(x=np.mean(residuals), color='green', linestyle='--', label=f'Mean: {np.mean(residuals):,.0f}')
axes[0, 1].set_title('Residual Distribution')
axes[0, 1].set_xlabel('Residual')
axes[0, 1].legend()

# 3. Actual vs Predicted
axes[1, 0].scatter(y_test, best_xgb_pred, alpha=0.5, s=20)
max_val = max(y_test.max(), best_xgb_pred.max())
axes[1, 0].plot([0, max_val], [0, max_val], 'r--', label='Perfect prediction')
axes[1, 0].set_title('Actual vs Predicted')
axes[1, 0].set_xlabel('Actual Sales')
axes[1, 0].set_ylabel('Predicted Sales')
axes[1, 0].legend()

# 4. Residuals vs Predicted (check for heteroscedasticity)
axes[1, 1].scatter(best_xgb_pred, residuals, alpha=0.5, s=20)
axes[1, 1].axhline(y=0, color='red', linestyle='--')
axes[1, 1].set_title('Residuals vs Predicted (Heteroscedasticity Check)')
axes[1, 1].set_xlabel('Predicted Sales')
axes[1, 1].set_ylabel('Residual')

plt.tight_layout()
plt.show()

# Residual statistics
print("="*50)
print("RESIDUAL STATISTICS")
print("="*50)
print(f"Mean residual: {np.mean(residuals):,.0f} (should be ~0)")
print(f"Std residual:  {np.std(residuals):,.0f}")
print(f"Min residual:  {np.min(residuals):,.0f}")
print(f"Max residual:  {np.max(residuals):,.0f}")

# Check for autocorrelation in residuals
from statsmodels.stats.diagnostic import acorr_ljungbox
lb_test = acorr_ljungbox(residuals, lags=[7, 14], return_df=True)
print(f"\nLjung-Box test for autocorrelation:")
print(lb_test)
print("\n(p-value > 0.05 suggests no significant autocorrelation)")

## 8. Model Comparison (vs Baseline)

Comparing advanced models against the baseline models from Phase 2.

In [ ]:
# ============================================
# MODEL COMPARISON
# ============================================

# Advanced model results
results_df = pd.DataFrame(results)

# Load baseline results if available
if baseline_path.exists():
    baseline_df = pd.read_csv(baseline_path)
    baseline_df['Type'] = 'Baseline'
    results_df['Type'] = 'Advanced'
    all_results = pd.concat([baseline_df, results_df], ignore_index=True)
else:
    results_df['Type'] = 'Advanced'
    all_results = results_df.copy()

# Sort by MAE
all_results = all_results.sort_values('MAE')

print("="*60)
print("COMPLETE MODEL COMPARISON")
print("="*60)
print(all_results.to_string(index=False))

# Identify champion
champion_model_name = all_results.iloc[0]['Model']
champion_mae = all_results.iloc[0]['MAE']
champion_type = all_results.iloc[0]['Type']

print(f"\n🏆 CHAMPION MODEL: {champion_model_name}")
print(f"   MAE: {champion_mae:,.0f} PLN")
print(f"   Type: {champion_type}")

# Improvement over baseline
if best_baseline_mae:
    improvement = (best_baseline_mae - champion_mae) / best_baseline_mae * 100
    print(f"\n📈 Improvement over best baseline: {improvement:.1f}%")

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Bar chart by MAE
colors = ['green' if t == 'Advanced' else 'steelblue' for t in all_results['Type']]
champion_idx = all_results[all_results['Model'] == champion_model_name].index[0]
colors[list(all_results.index).index(champion_idx)] = 'gold'

axes[0].barh(all_results['Model'], all_results['MAE'], color=colors)
axes[0].set_xlabel('MAE (PLN)')
axes[0].set_title('Model Comparison by MAE')
axes[0].invert_yaxis()

# Time series plot with champion prediction
axes[1].plot(test.index, y_test.values, 'k-', linewidth=2, label='Actual', alpha=0.8)
if champion_model_name == 'XGBoost (tuned)':
    champion_pred = best_xgb_pred
elif champion_model_name == 'XGBoost (default)':
    champion_pred = xgb_pred
elif champion_model_name == 'Prophet':
    champion_pred = prophet_pred
else:
    champion_pred = best_xgb_pred  # fallback
    
axes[1].plot(test.index, champion_pred, 'g--', linewidth=2, label=f'{champion_model_name} (Champion)', alpha=0.8)
axes[1].set_title('Champion Model: Actual vs Predicted')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Sales (PLN)')
axes[1].legend()

plt.tight_layout()
plt.show()

## 9. MLflow Logging & Champion Model Selection

Logging experiments to MLflow for reproducibility and tracking.

In [ ]:
# ============================================
# MLFLOW LOGGING
# ============================================

# Set up MLflow
mlflow.set_experiment('Time_Series_Forecasting')

print("="*50)
print("LOGGING TO MLFLOW")
print("="*50)

# Log the tuned XGBoost model (likely champion)
with mlflow.start_run(run_name='XGBoost_Tuned_Champion'):
    # Log parameters
    mlflow.log_params(best_params)
    mlflow.log_param('n_features', len(features))
    mlflow.log_param('train_size', len(train))
    mlflow.log_param('test_size', len(test))
    
    # Log metrics
    mlflow.log_metric('mae', best_mae)
    mlflow.log_metric('rmse', best_rmse)
    mlflow.log_metric('mape', best_mape)
    
    if best_baseline_mae:
        mlflow.log_metric('improvement_over_baseline', improvement)
    
    # Log model
    mlflow.sklearn.log_model(best_xgb, 'model')
    
    # Log feature importance as artifact
    importance_df.to_csv(RESULTS_DIR / 'feature_importance.csv', index=False)
    mlflow.log_artifact(str(RESULTS_DIR / 'feature_importance.csv'))
    
    run_id = mlflow.active_run().info.run_id
    print(f"✅ Logged XGBoost (tuned) to MLflow")
    print(f"   Run ID: {run_id}")

# Log Prophet model
with mlflow.start_run(run_name='Prophet'):
    mlflow.log_param('yearly_seasonality', True)
    mlflow.log_param('weekly_seasonality', True)
    mlflow.log_metric('mae', prophet_mae)
    mlflow.log_metric('rmse', prophet_rmse)
    mlflow.log_metric('mape', prophet_mape)
    print(f"✅ Logged Prophet to MLflow")

## 10. Save Champion Model

Saving the champion model for production deployment.

In [ ]:
# ============================================
# SAVE CHAMPION MODEL
# ============================================

# Save XGBoost tuned model (champion)
champion_model_path = MODELS_DIR / "champion_model.pkl"
joblib.dump(best_xgb, champion_model_path)
print(f"✅ Saved champion model to: {champion_model_path}")

# Save model metadata
model_metadata = {
    'model_name': 'XGBoost (tuned)',
    'best_params': best_params,
    'features': features,
    'metrics': {
        'mae': best_mae,
        'rmse': best_rmse,
        'mape': best_mape
    },
    'train_date_range': {
        'start': str(train.index.min().date()),
        'end': str(train.index.max().date())
    },
    'test_date_range': {
        'start': str(test.index.min().date()),
        'end': str(test.index.max().date())
    }
}

import json
metadata_path = MODELS_DIR / "champion_model_metadata.json"
with open(metadata_path, 'w') as f:
    json.dump(model_metadata, f, indent=2)
print(f"✅ Saved model metadata to: {metadata_path}")

# Save all results
all_results.to_csv(RESULTS_DIR / 'all_model_results.csv', index=False)
print(f"✅ Saved all results to: {RESULTS_DIR / 'all_model_results.csv'}")

# Save anomaly data
anomaly_cols = ['sales', 'anomaly_isoforest', 'anomaly_zscore', 'anomaly_iqr', 'anomaly_any']
df[anomaly_cols].to_parquet(PROCESSED_DIR / 'anomalies_detected.parquet')
print(f"✅ Saved anomaly data to: {PROCESSED_DIR / 'anomalies_detected.parquet'}")

# Final summary
print("\n" + "="*60)
print("PHASE 3 COMPLETE - SUMMARY")
print("="*60)
print(f"""
📊 ANOMALY DETECTION
   - Isolation Forest: {df['anomaly_isoforest'].sum()} anomalies
   - Z-score: {df['anomaly_zscore'].sum()} anomalies
   - IQR: {df['anomaly_iqr'].sum()} anomalies

🔧 FEATURE ENGINEERING
   - {len(features)} features created
   - Top features: {', '.join(importance_df.head(5)['feature'].tolist())}

🏆 CHAMPION MODEL: XGBoost (tuned)
   - MAE: {best_mae:,.0f} PLN
   - RMSE: {best_rmse:,.0f} PLN
   - MAPE: {best_mape:.1f}%
""")

if best_baseline_mae:
    print(f"""📈 IMPROVEMENT OVER BASELINE
   - Best baseline MAE: {best_baseline_mae:,.0f} PLN
   - Champion MAE: {best_mae:,.0f} PLN
   - Improvement: {improvement:.1f}%
""")

print("""
📁 SAVED ARTIFACTS
   - models/champion_model.pkl
   - models/champion_model_metadata.json
   - results/all_model_results.csv
   - results/feature_importance.csv
   - Data/processed/anomalies_detected.parquet
""")